In [3]:
from doctest import DocFileCase
import yfinance as yf

# Choose an interval: '1d' (daily), '1wk' (weekly), '1mo' (monthly), etc.
interval = '1d'  # Change this to '1wk' or '1mo' as needed
symbol = "^GSPC"
# Download historical data for S&P 500 (^GSPC) with the chosen interval
df = yf.download(symbol, period="1y", interval=interval)

print(f"Downloaded S&P 500 data with interval: {interval}")
print(df)


/tmp/ipykernel_2700/4119031107.py:8: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(symbol, period="1y", interval=interval)
[*********************100%***********************]  1 of 1 completed

Downloaded S&P 500 data with interval: 1d
Price             Close         High          Low         Open      Volume
Ticker            ^GSPC        ^GSPC        ^GSPC        ^GSPC       ^GSPC
Date                                                                      
2024-12-05  6075.109863  6094.549805  6072.899902  6089.029785  4212020000
2024-12-06  6090.270020  6099.970215  6079.979980  6081.379883  3924830000
2024-12-09  6052.850098  6088.509766  6048.629883  6083.009766  4556460000
2024-12-10  6034.910156  6065.399902  6029.890137  6057.589844  4048410000
2024-12-11  6084.189941  6092.589844  6060.149902  6060.149902  4269950000
...                 ...          ...          ...          ...         ...
2025-11-28  6849.089844  6850.859863  6819.750000  6822.520020  2558540000
2025-12-01  6812.629883  6843.649902  6799.939941  6812.299805  4549370000
2025-12-02  6829.370117  6851.549805  6806.709961  6830.959961  4582290000
2025-12-03  6849.720215  6862.419922  6810.430176  6815.29

In [ ]:
from lib.line import Calculator


calculator = Calculator()






In [ ]:
# Calculate dots first (needed for 6-x methods)
dots_valid = calculator.calculate_dots(df, symbol)
print(f"Calculated {len(dots_valid)} valid dots")
print(f"\nLast 5 dots:")
print(dots_valid.tail())

# Collect all resistance and support lines
resistances = []
supports = []

# 5-2 lines
res_5_2 = calculator.get_5_2_resistance(df, symbol)
if res_5_2[0] is not None:
    resistances.append(("5/2 Resistance", res_5_2))
    print(f"\n5/2 Resistance: {res_5_2}")

sup_5_2 = calculator.get_5_2_support(df, symbol)
if sup_5_2[0] is not None:
    supports.append(("5/2 Support", sup_5_2))
    print(f"5/2 Support: {sup_5_2}")

# 5-1 lines
res_5_1 = calculator.get_5_1_resistance(df, symbol)
if res_5_1[0] is not None:
    resistances.append(("5/1 Resistance", res_5_1))
    print(f"5/1 Resistance: {res_5_1}")

sup_5_1 = calculator.get_5_1_support(df, symbol)
if sup_5_1[0] is not None:
    supports.append(("5/1 Support", sup_5_1))
    print(f"5/1 Support: {sup_5_1}")

# 5-3 lines
res_5_3 = calculator.get_5_3_resistance(df, symbol)
if res_5_3[0] is not None:
    resistances.append(("5/3 Resistance", res_5_3))
    print(f"5/3 Resistance: {res_5_3}")

sup_5_3 = calculator.get_5_3_support(df, symbol)
if sup_5_3[0] is not None:
    supports.append(("5/3 Support", sup_5_3))
    print(f"5/3 Support: {sup_5_3}")

# 5-9 lines
res_5_9 = calculator.get_5_9_resistance(df, symbol)
if res_5_9[0] is not None:
    resistances.append(("5/9 Resistance", res_5_9))
    print(f"5/9 Resistance: {res_5_9}")

sup_5_9 = calculator.get_5_9_support(df, symbol)
if sup_5_9[0] is not None:
    supports.append(("5/9 Support", sup_5_9))
    print(f"5/9 Support: {sup_5_9}")

# 6-1 lines (require dots)
if len(dots_valid) >= 3:
    res_6_1 = calculator.get_6_1_resistance(df, dots_valid['dots'], symbol)
    if res_6_1[0] is not None:
        resistances.append(("6/1 Resistance", res_6_1))
        print(f"6/1 Resistance: {res_6_1}")

    sup_6_1 = calculator.get_6_1_support(df, dots_valid['dots'], symbol)
    if sup_6_1[0] is not None:
        supports.append(("6/1 Support", sup_6_1))
        print(f"6/1 Support: {sup_6_1}")

# 6-5 lines (require dots)
if len(dots_valid) >= 3:
    res_6_5 = calculator.get_6_5_resistance(df, dots_valid['dots'], symbol)
    if res_6_5[0] is not None:
        resistances.append(("6/5 Resistance", res_6_5))
        print(f"6/5 Resistance: {res_6_5}")

    sup_6_5 = calculator.get_6_5_support(df, dots_valid['dots'], symbol)
    if sup_6_5[0] is not None:
        supports.append(("6/5 Support", sup_6_5))
        print(f"6/5 Support: {sup_6_5}")

# 6-7 lines (require dots)
if len(dots_valid) >= 3:
    res_6_7 = calculator.get_6_7_resistance(df, dots_valid['dots'], symbol)
    if res_6_7[0] is not None:
        resistances.append(("6/7 Resistance", res_6_7))
        print(f"6/7 Resistance: {res_6_7}")

    sup_6_7 = calculator.get_6_7_support(df, dots_valid['dots'], symbol)
    if sup_6_7[0] is not None:
        supports.append(("6/7 Support", sup_6_7))
        print(f"6/7 Support: {sup_6_7}")

# Summary
print(f"\n{'='*60}")
print(f"SUMMARY:")
print(f"{'='*60}")
print(f"Total Resistances: {len(resistances)}")
for name, (p1, p2) in resistances:
    print(f"  - {name}: Point1=({p1[0]}, {p1[1]:.2f}), Point2=({p2[0]}, {p2[1]:.2f})")

print(f"\nTotal Supports: {len(supports)}")
for name, (p1, p2) in supports:
    print(f"  - {name}: Point1=({p1[0]}, {p1[1]:.2f}), Point2=({p2[0]}, {p2[1]:.2f})")
